# Overview
- This notebook is meant to perform a simple analysis between the rental prices listed in property guru and the actual rental prices transacted and published by the URA.
- The source of URA data comes from their own API
- The source of property guru data comes from their website as of Dec 2023
- The analysis is performed at each condominium comparing average listing price for that property and the average rental transacted values for that property.


In [21]:
import pandas as pd

LISTINGS_FILENAME = 'parsed_csvs/listings_2022_12_13.csv'
URA_FILENAME = 'rentals_ura_bigquery.csv'
START_DATE = "2022-08-30"  # used to filter the rentals table from URA
MIN_SQFT = 700
MAX_SQFT = 1000

listings_df = pd.read_csv(LISTINGS_FILENAME)
ura_df = pd.read_csv(URA_FILENAME)

print('Type of apartment types')
listings_df.groupby('property_type').property_name.count().sort_values(ascending=False)

unique_properties_listing = set(listings_df.loc[lambda x: (x.property_type != "HDB Flat")]\
                                .property_name.str.upper().unique())
unique_properties_ura = set(ura_df.project_name.unique())

print('Prop Guru Listing amount of properties: ',len(unique_properties_listing))
print('URA History amount of properties: ',len(unique_properties_ura))




print("Amount of matching property names: ", len(unique_properties_listing & unique_properties_ura))



listings_not_in_ura = unique_properties_listing.difference(unique_properties_ura)

print("Exampled of not matching properties")


Type of apartment types
Prop Guru Listing amount of properties:  3092
URA History amount of properties:  2359
Amount of matching property names:  1305
Exampled of not matching properties


In [22]:
ura_df.head()

,project_id,rent,low_area_sqm,high_area_sqm,low_area_sqft,high_area_sqft,lease_date,no_of_bedrooms,street_name,project_name,district,property_type,longitude,latitude,id,category_name
0,1103,3700,0,100,0,1000,2022-10-01,99,JALAN GEMBIRA,MACPHERSON GARDEN ESTATE,13,2,103.880137,1.332807,2,Terrace House
1,1103,3400,0,100,0,1000,2022-10-01,99,JALAN GEMBIRA,MACPHERSON GARDEN ESTATE,13,2,103.880137,1.332807,2,Terrace House
2,1103,4200,0,100,0,1000,2022-10-01,99,JALAN GEMBIRA,MACPHERSON GARDEN ESTATE,13,2,103.880137,1.332807,2,Terrace House
3,868,3300,0,100,0,1000,2022-10-01,99,JALAN CHEGAR,THOMSON GARDEN ESTATE,20,2,103.838773,1.353841,2,Terrace House
4,180,13800,260,270,2800,2900,2022-10-01,5,AMBER ROAD,THE SEA VIEW,15,1,103.902149,1.301674,1,Non-landed Properties


In [18]:
listings_df.head()

,property_name,listing_href,price,floor_area,property_address,is_studio,is_room,property_type,currency,num_bedrooms,num_bathrooms,price_period,build_year,furnished,district_code
0,34 LENGKONG SATU,https://www.propertyguru.com.sg/listing/241651...,1500,180,NaN,False,True,Semi-Detached House,S$,NaN,NaN,/mo,NaN,Partially Furnished,D14
1,PARC ESTA,https://www.propertyguru.com.sg/listing/241321...,4800,732,822 Sims Avenue,False,False,New Project: 2022,S$,2.0,2.0,/mo,NaN,Partially Furnished,D14
2,PARC ESTA,https://www.propertyguru.com.sg/listing/241522...,5000,732,822 Sims Avenue,False,False,New Project: 2022,S$,2.0,2.0,/mo,NaN,Partially Furnished,D14
3,"PAYA LEBAR - SO GROOVY ! !! FEEL IT, LOVE IT",https://www.propertyguru.com.sg/listing/217792...,6660,819,"PAYA LEBAR ROAD, SIMS AVENUE, PAYA LEBAR MRT",False,False,Condominium,S$,2.0,2.0,/mo,NaN,Fully Furnished,D14
4,WATERBANK AT DAKOTA,https://www.propertyguru.com.sg/listing/241512...,9000,1141,76 Dakota Crescent,False,False,Condominium,S$,3.0,2.0,/mo,2013.0,NaN,D14


In [24]:
left_df = listings_df.loc[lambda x: ~(x.is_studio) 
                          & ~(x.is_room) & (x.property_type!="HDB Flat")
                          & (x.floor_area>= MIN_SQFT) & (x.floor_area<=MAX_SQFT)
                          
                          ]\
    [['property_name','price','floor_area','is_studio','is_room',
      'property_type',"num_bedrooms","district_code","build_year","furnished"]]
    
print(len(left_df))
left_df.head()



2074


,property_name,price,floor_area,is_studio,is_room,property_type,num_bedrooms,district_code,build_year,furnished
1,PARC ESTA,4800,732,False,False,New Project: 2022,2.0,D14,NaN,Partially Furnished
2,PARC ESTA,5000,732,False,False,New Project: 2022,2.0,D14,NaN,Partially Furnished
3,"PAYA LEBAR - SO GROOVY ! !! FEEL IT, LOVE IT",6660,819,False,False,Condominium,2.0,D14,NaN,Fully Furnished
8,PARC ESTA,5300,710,False,False,New Project: 2022,2.0,D14,NaN,Partially Furnished
12,PARC ESTA,5797,840,False,False,New Project: 2022,3.0,D14,NaN,Partially Furnished


In [26]:

right_df = ura_df.loc[lambda x: (x.lease_date >= START_DATE) & (x.low_area_sqft >= MIN_SQFT) &
                      (x.high_area_sqft <= MAX_SQFT)]\
        [['project_name','rent','low_area_sqft','high_area_sqft','lease_date','no_of_bedrooms']]
right_df


,project_name,rent,low_area_sqft,high_area_sqft,lease_date,no_of_bedrooms
1580,HIGH PARK RESIDENCES,3100,700,800,2022-10-01,2
1581,J GATEWAY,4300,700,800,2022-10-01,2
1582,D'NEST,3300,700,800,2022-10-01,2
1583,EASTERN LAGOON,2400,700,800,2022-10-01,1
1584,SKY HABITAT,4000,700,800,2022-10-01,1
...,...,...,...,...,...,...
10683,ALTEZ,6800,900,1000,2022-09-01,2
10684,REGENTVILLE,3800,900,1000,2022-09-01,2
10685,GEM RESIDENCES,2450,900,1000,2022-09-01,3
10686,76 SHENTON,6200,900,1000,2022-09-01,2


In [31]:
left_df_grouped = left_df.groupby('property_name')['price'].mean()
right_df_grouped = right_df.groupby('project_name')['rent'].mean()
merged = pd.merge(left_df_grouped,right_df_grouped,
                  how='inner',left_index=True,right_index=True)

print('number of matches: ', len(merged))
merged.head()

number of matches:  375


,price,rent
10 SHELFORD,4000.0,3500.0
18 WOODSVILLE,5250.0,3310.0
1919,6900.0,3900.0
2 RVG,5650.0,5500.0
26 NEWTON,5987.5,5600.0


In [34]:
merged['diff'] = merged['price'] - merged['rent']
merged['diff_pct'] = 100 * (merged['price'] / merged['rent'] -1)
merged_sorted = merged.sort_values('diff_pct')
merged_sorted

,price,rent,diff,diff_pct
HONG HENG MANSIONS,1300.000000,2300.000000,-1000.000000,-43.478261
SANT RITZ,2600.000000,3400.000000,-800.000000,-23.529412
BELLA CASITA,3950.000000,4700.000000,-750.000000,-15.957447
TANGLIN REGENCY,4200.000000,4800.000000,-600.000000,-12.500000
LE QUEST,4260.000000,4600.000000,-340.000000,-7.391304
...,...,...,...,...
THE PLAZA,6533.333333,3157.142857,3376.190476,106.938160
LANGSTON VILLE,7666.666667,3650.000000,4016.666667,110.045662
RIVERSOUND RESIDENCE,4620.000000,2175.000000,2445.000000,112.413793
NINE RESIDENCES,4900.000000,2200.000000,2700.000000,122.727273


In [35]:
merged_sorted.describe()

,price,rent,diff,diff_pct
count,375.000000,375.000000,375.000000,375.000000
mean,5087.938200,4032.163879,1055.774321,28.793071
std,1142.243223,1071.667298,680.242274,21.684411
min,1300.000000,2175.000000,-1000.000000,-43.478261
25%,4300.000000,3350.000000,653.125000,15.252773
50%,4850.000000,3832.692308,979.861111,25.373134
75%,5702.916667,4483.333333,1382.134183,37.133788
max,14525.000000,13000.000000,4545.454545,153.846154
